In [1]:
import torchvision,torch,time
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential

# 设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))
import os
current_directory = os.getcwd()
print("Current working directory:", current_directory)



using cuda device.
Current working directory: d:\CODE\vscode\AI-learning\scripts


In [3]:

dataset_transform = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.FashionMNIST(root="../dataset", train=True, transform=dataset_transform, download=False)
test_data = torchvision.datasets.FashionMNIST(root="../dataset", train=False, transform=dataset_transform, download=False)
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)
# length 长度
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

训练数据集的长度为：60000
测试数据集的长度为：10000


In [4]:
# 创建网络模型    
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.model1 = Sequential(
            Conv2d(1, 32, 5, padding=4),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model1(x)
        return x
    
    
tudui = Tudui()
tudui = tudui.to(device)
loss_fn = nn.CrossEntropyLoss() # 交叉熵损失函数
loss_fn = loss_fn.to(device)
learning_rate = 1e-2
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)

total_train_step = 0    # 训练次数
total_test_step = 0     # 测试次数
epoch = 10              # 训练轮数
writer = SummaryWriter("../logs/hello_train")

In [6]:
last_time = time.time()
for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i+1))
    # 训练
    tudui.train() # 对某些特定的层需要此句
    for data in train_dataloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = tudui(inputs)
        loss = loss_fn(outputs, labels)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试
    tudui.eval() # 对某些特定的层需要此句
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = tudui(inputs)
            loss = loss_fn(outputs, labels)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == labels).sum() # argmax(0)竖着看，argmax(1)横着看
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    # torch.save(tudui, "../models/tudui_{}.pth".format(i))
    torch.save(tudui.state_dict(), "../models/hello/tudui_{}.pth".format(i))
    delta_time = time.time() - last_time
    last_time = time.time()
    print("模型已保存，耗时：{}".format(delta_time))

writer.close()

-------第 1 轮训练开始-------
训练次数：1000, Loss: 0.965538740158081
训练次数：1100, Loss: 0.4786267876625061
训练次数：1200, Loss: 0.743501603603363
训练次数：1300, Loss: 0.5958795547485352
训练次数：1400, Loss: 0.6995118260383606
训练次数：1500, Loss: 0.42427384853363037
训练次数：1600, Loss: 0.6376951932907104
训练次数：1700, Loss: 0.7919732928276062
训练次数：1800, Loss: 0.402107834815979
整体测试集上的Loss: 94.57190677523613
整体测试集上的正确率: 0.7795999646186829
模型已保存，耗时：5.128469705581665
-------第 2 轮训练开始-------
训练次数：1900, Loss: 0.5753287672996521
训练次数：2000, Loss: 0.764788806438446
训练次数：2100, Loss: 0.5603244304656982
训练次数：2200, Loss: 0.6376181840896606
训练次数：2300, Loss: 0.5137832760810852
训练次数：2400, Loss: 0.6801995038986206
训练次数：2500, Loss: 0.5052745938301086
训练次数：2600, Loss: 0.49501582980155945
训练次数：2700, Loss: 0.5301956534385681
训练次数：2800, Loss: 0.423621267080307
整体测试集上的Loss: 84.33061051368713
整体测试集上的正确率: 0.8030999898910522
模型已保存，耗时：5.011016607284546
-------第 3 轮训练开始-------
训练次数：2900, Loss: 0.5876094102859497
训练次数：3000, Loss: 0.81435281038284